In [1]:
%run load_data.ipynb
print (dir())

['DataFrame', 'In', 'Math', 'Out', 'SIZE_RATIO_CUT', 'SNR_CUT', 'Series', 'T_CUT', 'Table', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'bottleneck', 'display', 'exit', 'filter_by_snr', 'fits', 'get_df_list', 'get_ipython', 'h5py', 'i', 'index_2_3', 'index_2_3_true', 'index_2_3_true_negative', 'index_2_3_true_positive', 'index_4_5', 'index_4_5_true', 'index_4_5_true_negative', 'index_4_5_true_positive', 'load_mcal', 'load_true', 'mcal_df_list', 'mcal_df_list_negative', 'mcal_df_list_positive', 'np', 'numexpr', 'pd', 'photdf', 'plt', 'quit', 'tqdm', 'true_df_list']


In [2]:
import healpy as hp
import astropy_healpix as a_hp

# Choose NSIDE

In [6]:
NSIDE=256 #or 512
NPIX=hp.nside2npix(NSIDE)
print ("The number of pixels is: {}.".format(NPIX))
ipix_index=np.arange(NPIX)

The number of pixels is: 786432.


In [7]:
final_df_list=[None]*4
for i,catalog in enumerate(mcal_df_list_positive):
    final_df_list[i]=pd.concat([catalog,photdf[['true_cm_g_1','true_cm_g_2']]],axis=1,join='inner')

In [122]:
def get_healpix(df):
    print ("The length of the df is: {}.".format(len(df)))
    print ("The ratio of the length of the catagory to the total pixel number is: {:.2%}".format(len(df)/NPIX))
    df['ipix']=df.apply(lambda x:hp.ang2pix(NSIDE,np.pi/2-np.deg2rad(x['dec']),np.deg2rad(x['ra']),nest=True),axis=1)
    print("The propotion of rendered objects is: {:.2%}".format(np.sum(df['ipix']!=0)/len(df)))
    print ("Resetting indexes")
    df_ipix=df.set_index('ipix')
    df_ipix_sorted=df_ipix.sort_index()
    return (df_ipix_sorted)

def group_ipix(df_ipix_sorted):
    number_mask=df_ipix_sorted.groupby('ipix').count()['R11']<10
    df_sorted_grouped=df_ipix_sorted.groupby('ipix').mean()
    df_sorted_grouped_masked=df_sorted_grouped[number_mask]
    print ("The mean of R11 is {}, max is {}, min is {}".format(df_sorted_grouped_masked['R11'].mean(),df_sorted_grouped_masked['R11'].max(),df_sorted_grouped_masked['R11'].min()))
    print ("The mean of R22 is {}, max is {}, min is {}".format(df_sorted_grouped_masked['R22'].mean(),df_sorted_grouped_masked['R22'].max(),df_sorted_grouped_masked['R22'].min()))
    R11 = df_sorted_grouped_masked['R11'].mean()
    R22 = df_sorted_grouped_masked['R22'].mean()
    df_sorted_grouped_masked['delta_g1']=df_sorted_grouped_masked.apply(lambda x:x['e_1']/R11-x['true_cm_g_1'],axis=1)
    df_sorted_grouped_masked['delta_g2']=df_sorted_grouped_masked.apply(lambda x:x['e_2']/R22-x['true_cm_g_2'],axis=1)
    print ("The propotion of averaged objects is: {:.2%}".format((len(df)-len(df_sorted_grouped_masked))/len(df)))
    print ("---------------------------------")
    return (df_sorted_grouped_masked)

In [123]:
ipix_sorted = [get_healpix(df) for df in final_df_list]

The length of the df is: 272818.
The ratio of the length of the catagory to the total pixel number is: 34.69%
The propotion of rendered objects is: 100.00%
Resetting indexes
The length of the df is: 272818.
The ratio of the length of the catagory to the total pixel number is: 34.69%
The propotion of rendered objects is: 100.00%
Resetting indexes
The length of the df is: 215145.
The ratio of the length of the catagory to the total pixel number is: 27.36%
The propotion of rendered objects is: 100.00%
Resetting indexes
The length of the df is: 215145.
The ratio of the length of the catagory to the total pixel number is: 27.36%
The propotion of rendered objects is: 100.00%
Resetting indexes


In [124]:
ipix_sorted_filtered_grouped=[group_ipix(df) for df in ipix_sorted]

The mean of R11 is 0.78644835142623, max is 13.67369349540634, min is -6.992034474310287
The mean of R22 is 0.7855622549926757, max is 10.63652140008755, min is -12.231064460480107


/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


The propotion of averaged objects is: 97.78%
---------------------------------
The mean of R11 is 0.7750157162403251, max is 17.03993360179426, min is -6.842691657484413
The mean of R22 is 0.7847042804545312, max is 21.176713510351416, min is -32.04549961794578
The propotion of averaged objects is: 97.78%
---------------------------------
The mean of R11 is 0.7722151369321218, max is 7.48179347176924, min is -26.84834620190807
The mean of R22 is 0.81033143757935, max is 17.67501272412899, min is -10.426019279742748
The propotion of averaged objects is: 97.37%
---------------------------------
The mean of R11 is 0.7722151369321218, max is 7.48179347176924, min is -26.84834620190807
The mean of R22 is 0.81033143757935, max is 17.67501272412899, min is -10.426019279742748
The propotion of averaged objects is: 97.37%
---------------------------------


In [125]:
def reduce_for_plot(df):
    df_filled=df[['delta_g1','delta_g2']].reindex(index=ipix_index,fill_value=hp.pixelfunc.UNSEEN)
    print("The propotion of filled pixels is: {}.".format(np.sum(df_filled['delta_g1']>0)/NPIX))
    return(df_filled)

In [126]:
hp_reduced_list=[reduce_for_plot(df) for df in ipix_sorted_filtered_grouped]

The propotion of filled pixels is: 0.002887725830078125.
The propotion of filled pixels is: 0.0029856363932291665.
The propotion of filled pixels is: 0.003734588623046875.
The propotion of filled pixels is: 0.003734588623046875.


# Compare maps

In [97]:
maps=h5py.File('/home/cz136/Y3_GOLD_2_2_1_maps.h5','r')['maps'] #no neighbor

In [108]:
list(maps.keys())

['det_frac',
 'ebv_planck13',
 'g',
 'hpix',
 'i',
 'r',
 'star_density_piffv23',
 'z']

In [153]:
maps['det_frac']

<HDF5 dataset "det_frac": shape (25169981,), type "<f8">

In [129]:
maps_ipix_dict={}
for key in maps.keys():
    print (key, len(maps[key]))
    if len(maps[key])>10:
        maps_ipix_dict[key]=np.array(maps[key])

det_frac 25169981
ebv_planck13 25169981
g 7
hpix 25169981
i 7
r 7
star_density_piffv23 25169981
z 7


In [130]:
maps_ipix_dict

{'det_frac': array([1.000e+00, 8.750e-01, 1.000e+00, ..., 9.375e-02, 8.125e-01,
        6.250e-02]),
 'ebv_planck13': array([1.074e-01, 1.094e-01, 1.040e-01, ..., 7.002e-02, 7.014e-02,
        7.014e-02]),
 'hpix': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 2.013e+08, 2.013e+08,
        2.013e+08]),
 'star_density_piffv23': array([5.000e+00, -1.637e+30, -1.637e+30, ..., 6.000e+00, 9.000e+00,
        -1.637e+30])}

In [137]:
downgraded_maps={}
for key,value in maps_ipix_dict.items():
    downgraded_maps[key]=hp.pixelfunc.ud_grade(value, NSIDE, order_in='NEST', order_out='NEST')

TypeError: bad number of pixels